# Data Cleansing


In [177]:
MIKE_ORIGIALS = r'C:\DS Project\Images\Mike\Original'
NEIL_ORIGINALS = r'C:\DS Project\Images\Neil\Original'
SET_SECONDS = 2

## Imports

In [178]:
import psycopg2
import db_config as creds
import os
import time
import datetime
from stat import * 

In [179]:
class theImg:
  def __init__(self, filename, createdDate):
    self.filename = filename
    self.createdDate = createdDate

## Database functions

In [180]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
         
    conn = psycopg2.connect(conn_string)
    
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

def disconnect(conn,cursor):
    
    conn.close()

In [181]:
# Connecting to DB
#conn, cursor = connect()

#insert_image = """
#    INSERT INTO images (filename) VALUES('UA502')       
#    """

# Execute SQL Command and commit to DB
#cursor.execute(insert_image)
#conn.commit()

# Disconnect from DB
#disconnect(conn, cursor)

# Mike

In [182]:
imgSet = 1
mikeImages = []

# loop round each image in the original folder
for subdir, dirs, files in os.walk(MIKE_ORIGIALS):
    
    for filename in files:
        
        filepath = subdir + os.sep + filename
        stat = os.stat(filepath)
        
        # create new image object with attributes
        # time.asctime(time.localtime(stat[ST_MTIME]))
        createdDate = datetime.datetime.fromtimestamp(stat[ST_MTIME])
        
        #print (filename,createdDate,stat[ST_MTIME],mod_timestamp)
        print (filename,createdDate)
        newImg = theImg(filename, createdDate)
        
        # check if image is > 2 secods ofset from first image in set
        if (len(mikeImages) > 0) and ( (createdDate-mikeImages[0].createdDate).total_seconds() >= SET_SECONDS ):
        
            # insert each of the images in the list into the database
            # AddSetToDB(mikeImages)
            print (imgSet,"-",mikeImages[0].createdDate,'-',len(mikeImages))
            
            # increment the set counter
            imgSet+=1
            
            # clear out the list
            mikeImages.clear()
            
        
        #set the image set 
        newImg.imgSet = imgSet    
            
        # add the image to the list
        mikeImages.append(newImg)
               
        
        
# add the last set to the data base     
print ('last',imgSet,"-",mikeImages[0].createdDate,'-',len(mikeImages))     
        
        
        

        
          

01Mk0005.JPG 2020-03-11 07:10:52
01Mk0006.JPG 2020-03-11 07:10:54
1 - 2020-03-11 07:10:52 - 1
01Mk0007.JPG 2020-03-11 07:10:56
2 - 2020-03-11 07:10:54 - 1
01Mk0008.JPG 2020-03-11 07:10:58
3 - 2020-03-11 07:10:56 - 1
01Mk0009.JPG 2020-03-11 07:11:02
4 - 2020-03-11 07:10:58 - 1
01Mk0010.JPG 2020-03-11 07:11:02
01Mk0011.JPG 2020-03-11 07:11:02
01Mk0012.JPG 2020-03-11 07:11:02
01Mk0013.JPG 2020-03-11 07:11:04
5 - 2020-03-11 07:11:02 - 4
01Mk0014.JPG 2020-03-11 07:11:04
01Mk0015.JPG 2020-03-11 07:11:04
01Mk0016.JPG 2020-03-11 07:11:06
6 - 2020-03-11 07:11:04 - 3
01Mk0017.JPG 2020-03-11 07:11:06
01Mk0018.JPG 2020-03-11 07:11:06
01Mk0019.JPG 2020-03-11 07:11:06
01Mk0020.JPG 2020-03-11 07:11:08
7 - 2020-03-11 07:11:06 - 4
01Mk0021.JPG 2020-03-11 07:11:08
01Mk0022.JPG 2020-03-11 07:11:08
01Mk0023.JPG 2020-03-11 07:11:08
01Mk0024.JPG 2020-03-11 07:11:08
01Mk0025.JPG 2020-03-11 07:11:10
8 - 2020-03-11 07:11:08 - 5
01Mk0026.JPG 2020-03-11 07:11:12
9 - 2020-03-11 07:11:10 - 1
01Mk0027.JPG 2020-03-1

01Mk0420.JPG 2020-03-11 17:36:50
01Mk0421.JPG 2020-03-11 17:49:12
205 - 2020-03-11 17:36:50 - 2
01Mk0422.JPG 2020-03-11 17:49:14
206 - 2020-03-11 17:49:12 - 1
01Mk0423.JPG 2020-03-11 20:11:06
207 - 2020-03-11 17:49:14 - 1
01Mk0424.JPG 2020-03-11 20:11:06
01Mk0425.JPG 2020-03-11 20:11:08
208 - 2020-03-11 20:11:06 - 2
01Mk0426.JPG 2020-03-11 23:24:44
209 - 2020-03-11 20:11:08 - 1
01Mk0427.JPG 2020-03-12 00:33:50
210 - 2020-03-11 23:24:44 - 1
01Mk0428.JPG 2020-03-12 04:52:14
211 - 2020-03-12 00:33:50 - 1
01Mk0429.JPG 2020-03-12 04:52:18
212 - 2020-03-12 04:52:14 - 1
01Mk0430.JPG 2020-03-12 04:57:44
213 - 2020-03-12 04:52:18 - 1
01Mk0431.JPG 2020-03-12 04:57:44
01Mk0432.JPG 2020-03-12 05:20:08
214 - 2020-03-12 04:57:44 - 2
01Mk0433.JPG 2020-03-12 05:20:10
215 - 2020-03-12 05:20:08 - 1
01Mk0434.JPG 2020-03-12 06:47:06
216 - 2020-03-12 05:20:10 - 1
01Mk0435.JPG 2020-03-12 06:47:06
01Mk0436.JPG 2020-03-12 06:47:22
217 - 2020-03-12 06:47:06 - 2
01Mk0437.JPG 2020-03-12 06:47:22
01Mk0438.JPG 202

01Mk0814.JPG 2020-03-12 15:10:56
01Mk0815.JPG 2020-03-12 15:10:56
01Mk0816.JPG 2020-03-12 15:10:56
01Mk0817.JPG 2020-03-12 15:10:56
01Mk0818.JPG 2020-03-12 15:11:48
415 - 2020-03-12 15:10:56 - 5
01Mk0819.JPG 2020-03-12 15:11:48
01Mk0820.JPG 2020-03-12 15:11:48
01Mk0821.JPG 2020-03-12 15:11:50
416 - 2020-03-12 15:11:48 - 3
01Mk0822.JPG 2020-03-12 15:14:34
417 - 2020-03-12 15:11:50 - 1
01Mk0823.JPG 2020-03-12 15:14:36
418 - 2020-03-12 15:14:34 - 1
01Mk0824.JPG 2020-03-12 15:19:20
419 - 2020-03-12 15:14:36 - 1
01Mk0825.JPG 2020-03-12 15:19:20
01Mk0826.JPG 2020-03-12 15:19:20
01Mk0827.JPG 2020-03-12 15:19:20
01Mk0828.JPG 2020-03-12 15:19:22
420 - 2020-03-12 15:19:20 - 4
01Mk0829.JPG 2020-03-12 15:22:08
421 - 2020-03-12 15:19:22 - 1
01Mk0830.JPG 2020-03-12 15:35:00
422 - 2020-03-12 15:22:08 - 1
01Mk0831.JPG 2020-03-12 15:35:00
01Mk0832.JPG 2020-03-12 15:38:26
423 - 2020-03-12 15:35:00 - 2
01Mk0833.JPG 2020-03-12 15:38:26
01Mk0834.JPG 2020-03-12 15:38:26
01Mk0835.JPG 2020-03-12 15:43:46
424 

01Mk1155.JPG 2020-03-13 10:33:42
585 - 2020-03-13 10:33:40 - 1
01Mk1156.JPG 2020-03-13 10:33:44
586 - 2020-03-13 10:33:42 - 1
01Mk1157.JPG 2020-03-13 10:33:46
587 - 2020-03-13 10:33:44 - 1
01Mk1158.JPG 2020-03-13 10:33:50
588 - 2020-03-13 10:33:46 - 1
01Mk1159.JPG 2020-03-13 10:33:52
589 - 2020-03-13 10:33:50 - 1
01Mk1160.JPG 2020-03-13 10:33:54
590 - 2020-03-13 10:33:52 - 1
01Mk1161.JPG 2020-03-13 10:34:24
591 - 2020-03-13 10:33:54 - 1
01Mk1162.JPG 2020-03-13 10:34:28
592 - 2020-03-13 10:34:24 - 1
01Mk1163.JPG 2020-03-13 10:34:30
593 - 2020-03-13 10:34:28 - 1
01Mk1164.JPG 2020-03-13 10:34:30
01Mk1165.JPG 2020-03-13 10:34:32
594 - 2020-03-13 10:34:30 - 2
01Mk1166.JPG 2020-03-13 10:34:34
595 - 2020-03-13 10:34:32 - 1
01Mk1167.JPG 2020-03-13 10:34:36
596 - 2020-03-13 10:34:34 - 1
01Mk1168.JPG 2020-03-13 10:34:36
01Mk1169.JPG 2020-03-13 10:34:36
01Mk1170.JPG 2020-03-13 10:34:36
01Mk1171.JPG 2020-03-13 10:34:44
597 - 2020-03-13 10:34:36 - 4
01Mk1172.JPG 2020-03-13 10:34:44
01Mk1173.JPG 202

01Mk1527.JPG 2020-03-14 08:32:06
774 - 2020-03-14 08:27:44 - 2
01Mk1528.JPG 2020-03-14 08:33:20
775 - 2020-03-14 08:32:06 - 1
01Mk1529.JPG 2020-03-14 08:33:20
01Mk1530.JPG 2020-03-14 08:33:20
01Mk1531.JPG 2020-03-14 08:33:20
01Mk1532.JPG 2020-03-14 08:34:20
776 - 2020-03-14 08:33:20 - 4
01Mk1533.JPG 2020-03-14 08:37:30
777 - 2020-03-14 08:34:20 - 1
01Mk1534.JPG 2020-03-14 08:37:30
01Mk1535.JPG 2020-03-14 08:38:52
778 - 2020-03-14 08:37:30 - 2
01Mk1536.JPG 2020-03-14 08:38:52
01Mk1537.JPG 2020-03-14 08:39:14
779 - 2020-03-14 08:38:52 - 2
01Mk1538.JPG 2020-03-14 08:39:16
780 - 2020-03-14 08:39:14 - 1
01Mk1539.JPG 2020-03-14 08:53:22
781 - 2020-03-14 08:39:16 - 1
01Mk1540.JPG 2020-03-14 08:53:22
01Mk1541.JPG 2020-03-14 08:59:14
782 - 2020-03-14 08:53:22 - 2
01Mk1542.JPG 2020-03-14 08:59:14
01Mk1543.JPG 2020-03-14 09:14:34
783 - 2020-03-14 08:59:14 - 2
01Mk1544.JPG 2020-03-14 09:14:34
01Mk1545.JPG 2020-03-14 09:14:34
01Mk1546.JPG 2020-03-14 09:14:34
01Mk1547.JPG 2020-03-14 09:14:36
784 - 2

...

15964 - 2020-07-24 12:09:44 - 1
08Mk5222.JPG 2020-07-24 12:10:00
15965 - 2020-07-24 12:09:58 - 1
08Mk5223.JPG 2020-07-24 12:10:00
08Mk5224.JPG 2020-07-24 12:10:14
15966 - 2020-07-24 12:10:00 - 2
08Mk5225.JPG 2020-07-24 12:10:14
08Mk5226.JPG 2020-07-24 12:10:18
15967 - 2020-07-24 12:10:14 - 2
08Mk5227.JPG 2020-07-24 12:10:30
15968 - 2020-07-24 12:10:18 - 1
08Mk5228.JPG 2020-07-24 12:10:32
15969 - 2020-07-24 12:10:30 - 1
08Mk5229.JPG 2020-07-24 12:10:34
15970 - 2020-07-24 12:10:32 - 1
08Mk5230.JPG 2020-07-24 12:10:34
08Mk5231.JPG 2020-07-24 12:10:34
08Mk5232.JPG 2020-07-24 12:10:34
08Mk5233.JPG 2020-07-24 12:11:20
15971 - 2020-07-24 12:10:34 - 4
08Mk5234.JPG 2020-07-24 12:11:20
08Mk5235.JPG 2020-07-24 12:13:54
15972 - 2020-07-24 12:11:20 - 2
08Mk5236.JPG 2020-07-24 12:13:56
15973 - 2020-07-24 12:13:54 - 1
08Mk5237.JPG 2020-07-24 12:14:00
15974 - 2020-07-24 12:13:56 - 1
08Mk5238.JPG 2020-07-24 12:21:14
15975 - 2020-07-24 12:14:00 - 1
08Mk5239.JPG 2020-07-24 12:21:14
08Mk5240.JPG 2020-07-2

16095 - 2020-07-24 15:48:22 - 2
08Mk5463.JPG 2020-07-24 15:48:24
08Mk5464.JPG 2020-07-24 15:51:30
16096 - 2020-07-24 15:48:24 - 2
08Mk5465.JPG 2020-07-24 15:52:52
16097 - 2020-07-24 15:51:30 - 1
08Mk5466.JPG 2020-07-24 15:52:54
16098 - 2020-07-24 15:52:52 - 1
08Mk5467.JPG 2020-07-24 15:53:22
16099 - 2020-07-24 15:52:54 - 1
08Mk5468.JPG 2020-07-24 16:00:36
16100 - 2020-07-24 15:53:22 - 1
08Mk5469.JPG 2020-07-24 16:00:36
08Mk5470.JPG 2020-07-24 16:04:04
16101 - 2020-07-24 16:00:36 - 2
08Mk5471.JPG 2020-07-24 16:04:06
16102 - 2020-07-24 16:04:04 - 1
08Mk5472.JPG 2020-07-24 16:04:06
08Mk5473.JPG 2020-07-24 16:04:12
16103 - 2020-07-24 16:04:06 - 2
08Mk5474.JPG 2020-07-24 16:04:12
08Mk5475.JPG 2020-07-24 16:04:14
16104 - 2020-07-24 16:04:12 - 2
08Mk5476.JPG 2020-07-24 16:04:18
16105 - 2020-07-24 16:04:14 - 1
08Mk5477.JPG 2020-07-24 16:04:20
16106 - 2020-07-24 16:04:18 - 1
08Mk5478.JPG 2020-07-24 16:04:20
08Mk5479.JPG 2020-07-24 16:04:20
08Mk5480.JPG 2020-07-24 16:04:24
16107 - 2020-07-24 16:

08Mk5703.JPG 2020-07-25 07:47:34
16239 - 2020-07-25 07:42:26 - 1
08Mk5704.JPG 2020-07-25 07:52:12
16240 - 2020-07-25 07:47:34 - 1
08Mk5705.JPG 2020-07-25 07:52:14
16241 - 2020-07-25 07:52:12 - 1
08Mk5706.JPG 2020-07-25 07:54:58
16242 - 2020-07-25 07:52:14 - 1
08Mk5707.JPG 2020-07-25 07:54:58
08Mk5708.JPG 2020-07-25 07:54:58
08Mk5709.JPG 2020-07-25 07:55:00
16243 - 2020-07-25 07:54:58 - 3
08Mk5710.JPG 2020-07-25 07:55:00
08Mk5711.JPG 2020-07-25 07:56:18
16244 - 2020-07-25 07:55:00 - 2
08Mk5712.JPG 2020-07-25 07:56:20
16245 - 2020-07-25 07:56:18 - 1
08Mk5713.JPG 2020-07-25 07:56:20
08Mk5714.JPG 2020-07-25 08:02:18
16246 - 2020-07-25 07:56:20 - 2
08Mk5715.JPG 2020-07-25 08:02:18
08Mk5716.JPG 2020-07-25 08:08:28
16247 - 2020-07-25 08:02:18 - 2
08Mk5717.JPG 2020-07-25 08:17:50
16248 - 2020-07-25 08:08:28 - 1
08Mk5718.JPG 2020-07-25 08:18:54
16249 - 2020-07-25 08:17:50 - 1
08Mk5719.JPG 2020-07-25 08:18:54
08Mk5720.JPG 2020-07-25 08:18:56
16250 - 2020-07-25 08:18:54 - 2
08Mk5721.JPG 2020-07-2

08Mk5987.JPG 2020-07-25 15:16:20
16403 - 2020-07-25 15:06:12 - 1
08Mk5988.JPG 2020-07-25 15:16:20
08Mk5989.JPG 2020-07-25 15:16:22
16404 - 2020-07-25 15:16:20 - 2
08Mk5990.JPG 2020-07-25 15:16:26
16405 - 2020-07-25 15:16:22 - 1
08Mk5991.JPG 2020-07-25 15:16:28
16406 - 2020-07-25 15:16:26 - 1
08Mk5992.JPG 2020-07-25 15:16:28
08Mk5993.JPG 2020-07-25 15:16:28
08Mk5994.JPG 2020-07-25 15:16:30
16407 - 2020-07-25 15:16:28 - 3
08Mk5995.JPG 2020-07-25 15:16:32
16408 - 2020-07-25 15:16:30 - 1
08Mk5996.JPG 2020-07-25 15:16:32
08Mk5997.JPG 2020-07-25 15:16:34
16409 - 2020-07-25 15:16:32 - 2
08Mk5998.JPG 2020-07-25 15:16:34
08Mk5999.JPG 2020-07-25 15:16:36
16410 - 2020-07-25 15:16:34 - 2
08Mk6000.JPG 2020-07-25 15:16:36
08Mk6001.JPG 2020-07-25 15:16:36
08Mk6002.JPG 2020-07-25 15:16:36
08Mk6003.JPG 2020-07-25 15:16:40
16411 - 2020-07-25 15:16:36 - 4
08Mk6004.JPG 2020-07-25 15:19:22
16412 - 2020-07-25 15:16:40 - 1
08Mk6005.JPG 2020-07-25 15:19:22
08Mk6006.JPG 2020-07-25 15:19:24
16413 - 2020-07-25 1

16586 - 2020-07-26 09:53:44 - 1
08Mk6292.JPG 2020-07-26 09:53:46
08Mk6293.JPG 2020-07-26 09:54:14
16587 - 2020-07-26 09:53:46 - 2
08Mk6294.JPG 2020-07-26 09:54:16
16588 - 2020-07-26 09:54:14 - 1
08Mk6295.JPG 2020-07-26 09:57:08
16589 - 2020-07-26 09:54:16 - 1
08Mk6296.JPG 2020-07-26 09:57:08
08Mk6297.JPG 2020-07-26 09:57:10
16590 - 2020-07-26 09:57:08 - 2
08Mk6298.JPG 2020-07-26 09:58:36
16591 - 2020-07-26 09:57:10 - 1
08Mk6299.JPG 2020-07-26 09:58:38
16592 - 2020-07-26 09:58:36 - 1
08Mk6300.JPG 2020-07-26 09:58:38
08Mk6301.JPG 2020-07-26 09:58:40
16593 - 2020-07-26 09:58:38 - 2
08Mk6302.JPG 2020-07-26 09:59:22
16594 - 2020-07-26 09:58:40 - 1
08Mk6303.JPG 2020-07-26 09:59:22
08Mk6304.JPG 2020-07-26 09:59:22
08Mk6305.JPG 2020-07-26 10:01:58
16595 - 2020-07-26 09:59:22 - 3
08Mk6306.JPG 2020-07-26 10:02:00
16596 - 2020-07-26 10:01:58 - 1
08Mk6307.JPG 2020-07-26 10:02:00
08Mk6308.JPG 2020-07-26 10:02:00
08Mk6309.JPG 2020-07-26 10:02:02
16597 - 2020-07-26 10:02:00 - 3
08Mk6310.JPG 2020-07-2